# Cython Testing

Testing compilation and running of cython module

##Notes
- Copied from labeyrieClasses : 48s
- With all extra fluff and if statements removed: 47s
- Casting to np.float32 instead of float: 22.9s

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft2,ifft2,fftshift
from labeyrieClasses import target

%matplotlib inline
%load_ext cython

In [2]:
# Comparison of bare Python to Cython for preprocessing FITS data
binary = target()

In [3]:
# Import demo .FITS file
binary.fits.fileName = "/home/niels/Documents/FITS/KP330.fits"
binary.fits.read(numDimensions=3)

Filename: /home/niels/Documents/FITS/KP330.fits
No.    Name         Type      Cards   Dimensions   Format
0    PRIMARY     PrimaryHDU      62   (512, 512, 1000)   float32   
None


In [4]:
%%timeit -n 1 -r 1
binary.fits.read(numDimensions=3)
# Preprocess .FITS with Python, time it
# Command copied from labeyrieClasses

if (len(binary.fits.data.shape) == 3):
    # Generate empty array the size of an image to be used to accumulate
    #  PSD values before averaging.
    psdSum = np.zeros(binary.fits.data.shape[1:3])

    imgNum = np.shape(binary.fits.data)[0] # Number of images
    imgIncrement = imgNum/20 # How often to display a status message

    # Looping through all images in cube
    for index,img in enumerate(binary.fits.data):

        # Print current file being processed
        if (((index+1) % imgIncrement) == 0):
            print("Processed Image #: ",(index+1),"/",imgNum)

        # FFT function requires little-endian data, so casting it
        img = img.astype(float)

        # Calculate 2D power spectrum
        # This gives us only real values
        psdImg = np.abs(fft2(img))**2

        # Accumulate current PSD value
        psdSum = np.add(psdSum,psdImg)

    # Divide by # of images to calculate average
    psdAvg = np.divide(psdSum,imgNum)

    # Normalizing FFT
    psdAvg = np.divide(psdAvg, (psdAvg.size)**2)

#Otherwise if FITS data is only one image
elif (len(binary.fits.shape) == 2):
    # FFT function requires little-endian data, so casting it
    img = binary.fits.astype(float)

    # Calculate 2D power spectrum
    # This gives us only real values
    psdImg = np.abs(fft2(img))**2

    # Normalizing FFT
    psdAvg = np.divide(psdImg, (psdImg.size)**2)

    binary.psd.data = fftshift(psdAvg)

Filename: /home/niels/Documents/FITS/KP330.fits
No.    Name         Type      Cards   Dimensions   Format
0    PRIMARY     PrimaryHDU      62   (512, 512, 1000)   float32   
None
Processed Image #:  50 / 1000
Processed Image #:  100 / 1000
Processed Image #:  150 / 1000
Processed Image #:  200 / 1000
Processed Image #:  250 / 1000
Processed Image #:  300 / 1000
Processed Image #:  350 / 1000
Processed Image #:  400 / 1000
Processed Image #:  450 / 1000
Processed Image #:  500 / 1000
Processed Image #:  550 / 1000
Processed Image #:  600 / 1000
Processed Image #:  650 / 1000
Processed Image #:  700 / 1000
Processed Image #:  750 / 1000
Processed Image #:  800 / 1000
Processed Image #:  850 / 1000
Processed Image #:  900 / 1000
Processed Image #:  950 / 1000
Processed Image #:  1000 / 1000
1 loops, best of 1: 48.4 s per loop


In [5]:
%%timeit -n 1 -r 1
binary.fits.read(numDimensions=3)
# Preprocess .FITS with Python, time it
# Command copied from labeyrieClasses
# Modified with all "if" statements removed

psdSum = np.zeros(binary.fits.data.shape[1:3])

imgNum = np.shape(binary.fits.data)[0] # Number of images
imgIncrement = imgNum/20 # How often to display a status message

# Looping through all images in cube
for index,img in enumerate(binary.fits.data):

    # FFT function requires little-endian data, so casting it
    img = img.astype(float)

    # Calculate 2D power spectrum
    # This gives us only real values
    psdImg = np.abs(fft2(img))**2

    # Accumulate current PSD value
    psdSum = np.add(psdSum,psdImg)

# Divide by # of images to calculate average
psdAvg = np.divide(psdSum,imgNum)

# Normalizing FFT
psdAvg = np.divide(psdAvg, (psdAvg.size)**2)

Filename: /home/niels/Documents/FITS/KP330.fits
No.    Name         Type      Cards   Dimensions   Format
0    PRIMARY     PrimaryHDU      62   (512, 512, 1000)   float32   
None
1 loops, best of 1: 47.5 s per loop


In [6]:
%%timeit -n 1 -r 1
binary.fits.read(numDimensions=3)
# Preprocess .FITS with Python, time it
# Command copied from labeyrieClasses
# Modified with all "if" statements removed

psdSum = np.zeros(binary.fits.data.shape[1:3])

imgNum = np.shape(binary.fits.data)[0] # Number of images
imgIncrement = imgNum/20 # How often to display a status message

# Looping through all images in cube
for index,img in enumerate(binary.fits.data):

    # FFT function requires little-endian data, so casting it
    img = img.astype(np.float32)

    # Calculate 2D power spectrum
    # This gives us only real values
    psdImg = np.abs(fft2(img))**2

    # Accumulate current PSD value
    psdSum = np.add(psdSum,psdImg)

# Divide by # of images to calculate average
psdAvg = np.divide(psdSum,imgNum)

# Normalizing FFT
psdAvg = np.divide(psdAvg, (psdAvg.size)**2)

Filename: /home/niels/Documents/FITS/KP330.fits
No.    Name         Type      Cards   Dimensions   Format
0    PRIMARY     PrimaryHDU      62   (512, 512, 1000)   float32   
None
1 loops, best of 1: 22.9 s per loop
